In [55]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate, Reshape
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')
from keras.models import load_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
first_model = load_model('/content/drive/MyDrive/Final Election Project/2002_model.h5')
df_2008 = pd.read_csv('/content/drive/MyDrive/Final Election Project/Complete 2008 new.csv')

In [57]:
def process_csv(file_path):
    # Read the CSV file

    # Extract the 'contituency' column
    constituency_column = df_2008['constituency_number']

    # Convert constituency numbers to a set to get unique values
    unique_constituencies = set(constituency_column)

    # Sort the unique constituency numbers in ascending order
    sorted_constituencies = sorted(unique_constituencies, key=lambda x: int(x.split('-')[1]))

    return sorted_constituencies

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/Final Election Project/Complete 2013.csv'
result = process_csv(file_path)
print(result)
# Complete list of NA numbers from 1 to 272
complete_na_list = ['NA-{}'.format(i) for i in range(1, 273)]

# Replace 'result' with the actual result obtained from the previous code
existing_na_list = result

# Find missing NA numbers
missing_na_list = [na for na in complete_na_list if na not in existing_na_list]

# Print the missing NA numbers
print("Missing NA numbers:", missing_na_list)


['NA-1', 'NA-2', 'NA-3', 'NA-4', 'NA-5', 'NA-6', 'NA-7', 'NA-8', 'NA-9', 'NA-10', 'NA-11', 'NA-12', 'NA-13', 'NA-14', 'NA-15', 'NA-16', 'NA-17', 'NA-18', 'NA-19', 'NA-20', 'NA-21', 'NA-22', 'NA-23', 'NA-24', 'NA-25', 'NA-26', 'NA-27', 'NA-28', 'NA-29', 'NA-30', 'NA-31', 'NA-32', 'NA-33', 'NA-34', 'NA-35', 'NA-36', 'NA-37', 'NA-38', 'NA-39', 'NA-40', 'NA-41', 'NA-42', 'NA-43', 'NA-44', 'NA-45', 'NA-46', 'NA-47', 'NA-48', 'NA-49', 'NA-50', 'NA-51', 'NA-52', 'NA-53', 'NA-54', 'NA-55', 'NA-56', 'NA-57', 'NA-58', 'NA-59', 'NA-60', 'NA-61', 'NA-62', 'NA-63', 'NA-64', 'NA-65', 'NA-66', 'NA-67', 'NA-68', 'NA-69', 'NA-70', 'NA-71', 'NA-72', 'NA-73', 'NA-74', 'NA-75', 'NA-76', 'NA-77', 'NA-78', 'NA-79', 'NA-80', 'NA-81', 'NA-82', 'NA-83', 'NA-84', 'NA-85', 'NA-86', 'NA-87', 'NA-88', 'NA-89', 'NA-90', 'NA-91', 'NA-92', 'NA-93', 'NA-94', 'NA-95', 'NA-96', 'NA-97', 'NA-98', 'NA-99', 'NA-100', 'NA-101', 'NA-102', 'NA-103', 'NA-104', 'NA-105', 'NA-106', 'NA-107', 'NA-108', 'NA-109', 'NA-110', 'NA-111

In [58]:
df_2008['constituency_number'].unique()

array(['NA-1', 'NA-10', 'NA-100', 'NA-101', 'NA-102', 'NA-103', 'NA-104',
       'NA-105', 'NA-106', 'NA-107', 'NA-108', 'NA-109', 'NA-11',
       'NA-110', 'NA-111', 'NA-112', 'NA-113', 'NA-114', 'NA-115',
       'NA-116', 'NA-117', 'NA-118', 'NA-12', 'NA-120', 'NA-121',
       'NA-122', 'NA-123', 'NA-124', 'NA-125', 'NA-126', 'NA-127',
       'NA-128', 'NA-129', 'NA-13', 'NA-130', 'NA-131', 'NA-132',
       'NA-133', 'NA-134', 'NA-135', 'NA-136', 'NA-137', 'NA-138',
       'NA-139', 'NA-14', 'NA-140', 'NA-141', 'NA-142', 'NA-143',
       'NA-144', 'NA-145', 'NA-146', 'NA-147', 'NA-148', 'NA-149',
       'NA-15', 'NA-150', 'NA-151', 'NA-152', 'NA-153', 'NA-154',
       'NA-155', 'NA-156', 'NA-157', 'NA-158', 'NA-159', 'NA-16',
       'NA-160', 'NA-161', 'NA-162', 'NA-163', 'NA-164', 'NA-165',
       'NA-166', 'NA-167', 'NA-168', 'NA-169', 'NA-17', 'NA-170',
       'NA-171', 'NA-172', 'NA-173', 'NA-174', 'NA-175', 'NA-176',
       'NA-177', 'NA-178', 'NA-179', 'NA-18', 'NA-180', 'NA-18

In [59]:
len(df_2008['constituency_number'].unique())

272

In [60]:
desire_col = ['constituency_number','province','candidate_name','candidate_party','voter_reg','outcome']
data = df_2008[desire_col]

In [61]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [62]:
def tokenize_and_pad(data, max_len):
    sequences = pad_sequences(data, maxlen=max_len, padding='post')
    return sequences.tolist()

In [63]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-63-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-63-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [64]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'voter_reg' ]]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [65]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [66]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded','voter_reg']], processed_data['outcome'], test_size=0.2, random_state=42)

In [156]:
'''model = Sequential()
model.add(first_model)
model.add(Reshape((-1, 1)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='softmax'))  # Use 'sigmoid' for binary classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])'''
model2 = Sequential()
model2.add(first_model)
model2.add(Reshape((-1, 1)))
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.3))
model2.add(LSTM(32, return_sequences=True))
model2.add(LSTM(32))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))  # Change activation function for binary classification
model2.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])


In [157]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

In [158]:
model2.fit(X_train_padded, y_train, epochs=5, validation_split=0.3)

Epoch 1/5
40/40 [==============================] - 22s 194ms/step - loss: 0.6931 - accuracy: 0.5642 - val_loss: 0.6931 - val_accuracy: 0.1077
Epoch 2/5
40/40 [==============================] - 4s 99ms/step - loss: 0.6931 - accuracy: 0.7371 - val_loss: 0.6931 - val_accuracy: 0.8905
Epoch 3/5
40/40 [==============================] - 5s 126ms/step - loss: 0.6930 - accuracy: 0.8318 - val_loss: 0.6930 - val_accuracy: 0.8905
Epoch 4/5
40/40 [==============================] - 4s 96ms/step - loss: 0.6929 - accuracy: 0.8701 - val_loss: 0.6929 - val_accuracy: 0.8905
Epoch 5/5
40/40 [==============================] - 4s 100ms/step - loss: 0.6928 - accuracy: 0.8725 - val_loss: 0.6927 - val_accuracy: 0.8905


In [152]:
#model2.save('/content/drive/MyDrive/Final Election Project/2008_model.h5')
model2.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 1)                 1350689   
                                                                 
 reshape_31 (Reshape)        (None, 1, 1)              0         
                                                                 
 lstm_106 (LSTM)             (None, 1, 64)             16896     
                                                                 
 dropout_43 (Dropout)        (None, 1, 64)             0         
                                                                 
 lstm_107 (LSTM)             (None, 1, 32)             12416     
                                                                 
 lstm_108 (LSTM)             (None, 32)                8320      
                                                                 
 dense_62 (Dense)            (None, 16)              

In [153]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model2.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

15/15 [==============================] - 0s 23ms/step - loss: 0.6382 - accuracy: 0.8818
Test Loss: 0.6382186412811279
Test Accuracy: 0.8818380832672119


In [159]:
predictions_2008 = model2.predict(X_test_padded)
predictions_2008

15/15 [==============================] - 3s 21ms/step


array([[0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.4997424 ],
       [0.499